# Cuaderno de ingesta de datos
En este bloque traeremos la informacipon desde datos abiertos.

In [2]:
import pandas as pd
from sodapy import Socrata

In [8]:
token = None  
codigo_dataset = "ch4w-5xir" 


In [11]:
from sodapy import Socrata
import pandas as pd

token = None  
codigo_dataset = "jbjy-vk9h"  # Dataset de ejemplo válido

client = Socrata("www.datos.gov.co", token, timeout=1000)
results = client.get(codigo_dataset, limit=1000)

df = pd.DataFrame.from_records(results)
print(df.head())



                             nombre_entidad nit_entidad  \
0   SENA REGIONAL CAUCA Centro agropecuario   899999034   
1              MUNICIPIO MANTA CUNDINAMARCA   800094711   
2                    MUNICIPIO DE SINCELEJO   800104062   
3                             CANAL CAPITAL   830012587   
4  GOBERNACION DE CALDAS - Primero la Gente   890801052   

                 departamento       ciudad                      localizaci_n  \
0                       Cauca      Popayán       Colombia,  Cauca ,  Popayán   
1                Cundinamarca        Manta  Colombia,  Cundinamarca ,  Manta   
2                       Sucre    Sincelejo     Colombia,  Sucre ,  Sincelejo   
3  Distrito Capital de Bogotá  No Definido     Colombia, Bogotá, No Definido   
4                      Caldas  No Definido   Colombia,  Caldas , No Definido   

         orden                  sector                  rama  \
0     Nacional                 Trabajo             Ejecutivo   
1  Territorial        Servicio Públi

In [ ]:
from pyspark.sql import SparkSession

# Crear o usar una sesión de Spark
spark = SparkSession.builder \
    .appName("Diplomado") \
    .getOrCreate()

# Ahora sí puedes ejecutar consultas
df = spark.sql("SHOW TABLES")  # Esto mostrará tablas si tienes un catálogo configurado
df.show()


In [ ]:
#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")

client = Socrata("www.datos.gov.co",str(token),timeout=1000)

In [ ]:
%sql
SHOW TABLES IN main.diplomado_datos;

In [ ]:
%sql
CREATE TABLE main.diplomado_datos.ids_contratos_procesos (
    nit_de_la_entidad STRING,
    otro_campo STRING
)
USING DELTA;

In [0]:
df_secop_id=spark.table("main.diplomado_datos.ids_contratos_procesos")
lista_entidades=df_secop_id.select("nit_de_la_entidad").distinct().collect()
len(list(map(lambda x: x.nit_de_la_entidad,lista_entidades)))

In [0]:
import time

mode_write = "overwrite"
reintentos = 5

for entidad in lista_entidades:
  intentos = 0
  while intentos < reintentos:
    try:
      query="""
      SELECT * WHERE nit_de_la_entidad = '{0}'
      """.format(entidad.nit_de_la_entidad)
      print(query)
      results_entidad=client.get(codigo_dataset,query=query)
      df_total=spark.createDataFrame(results_entidad)
      df_total.write \
        .format("delta") \
        .mode(mode_write) \
        .option("overwriteSchema", "true") \
        .saveAsTable("diplomado_datos.secop_bronze")
      mode_write = "append"
      break
    except Exception as e:
      intentos += 1
      time.sleep(10)
      print("Error: {0}".format(e))


print("Se cargaron todos los contratos")


In [0]:
df_secop_id=spark.table("main.diplomado_datos.ids_contratos_procesos")
df_secop_id.count()

In [0]:

# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import sha2, concat_ws, col

# 1. Cargar la tabla correcta desde el catálogo a un DataFrame
# Se asume que esta es la tabla que contiene las columnas que mencionaste.
df_secop_id = spark.table("main.diplomado_datos.ids_contratos_procesos")

# 2. Definir la lista corregida de columnas para el identificador único
columnas_para_hash = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor",
    "estado_del_proceso"
]

In [0]:

# 3. Añadir la nueva columna 'id_unico'
# Se concatenan las columnas clave con un separador y se les aplica un hash SHA-2.
df_con_id = df_secop_id.withColumn(
    "id_unico_con_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash]), 256)
)

columnas_para_hash_se = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor"
]
df_con_id = df_con_id.withColumn(
    "id_unico_sin_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash_se]), 256)
)
df_con_id.display()
     

In [0]:
df_con_id.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("diplomado_datos.secop_id_bronze")